there is a numpad to open a door, we'll call this the numeric_keypad
the numpad is a 4x3 grid, where the last row's first cell is empty (second row is 0, and last row is 'A')
you can operate the numpad using a robot
this robot is operated using a remote, we'll call this the directional_keypad
a directional_keypad is a 2x3 grid, the first row's first cell is empty, second is ^, then A. The second row is <, v, >
unfortunately the numeric_keypad-operating robot's (doorbot) remote is only accessible by a different robot
this robot is also operated by a directional_keypad
we'll call this robot the doorbotbot
in yet another twist, the doorbotbot's remote is only accessible by another robot
we'll call this robot the doorbotbotbot
the doorbotbotbot is operated by a directional_keypad, operated by you

directional_keypad:
```plaintext
_ ^ A
< v >
```

numeric_keypad:
```plaintext
7 8 9
4 5 6
1 2 3
_ 0 A
```

_ is the empty cell

initial state of any keypad is always 'A'

to summarize:
- you operate the doorbotbotbot using a directional_keypad
- the doorbotbotbot operates the doorbotbot using a directional_keypad
- the doorbotbot operates the doorbot using a directional_keypad
- the doorbot opens the door using a numeric_keypad

simply pressing the button 0 on the numeric_keypad will then require:
1. doorbot moves finger from A to 0
2. doorbotbot has pressed the sequence: <, A
3. doorbotbotbot has pressed the sequence: v, <, <, A, >, >, ^, A

you can see the pattern here, given a desired navigation from state to state, the spawning sequence of directional_keypad presses can be calculated

In [1]:
move_dict = {
    (0, 1): '>',
    (1, 0): 'v',
    (0, -1): '<',
    (-1, 0): '^',
}

move_dir_map = {
    (0, 1): 0,
    (1, 0): 1,
    (0, -1): 2,
    (-1, 0): 3
}

inv_move_dir_map = {v: k for k, v in move_dir_map.items()}

num_keypad = [['7', '8', '9'], ['4', '5', '6'], ['1', '2', '3'], ['_', '0', 'A']]
num_keypad_map = {num_keypad[i][j]: (i, j) for i in range(4) for j in range(3)}

directional_keypad = [['_', '^', 'A'], ['<', 'v', '>']]
directional_keypad_map = {directional_keypad[i][j]: (i, j) for i in range(2) for j in range(3)}

assert num_keypad_map['0'] == (3, 1), num_keypad_map
assert num_keypad_map['_'] == (3, 0), num_keypad_map
assert num_keypad_map['A'] == (3, 2), num_keypad_map
assert directional_keypad_map['^'] == (0, 1), directional_keypad_map
assert directional_keypad_map['A'] == (0, 2), directional_keypad_map

In [2]:
import heapq

class KeypadNode():
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    def __init__(self, pos: tuple[int, int], dir: int, val: str):
        self.pos = pos
        self.val = val
        self.dir = dir
        
    def __add__(self, move: tuple[int, int]) -> tuple[int, int]:
        return self.pos[0] + move[0], self.pos[1] + move[1]
    
    def __eq__(self, other):
        # if other is not a node, return false
        if not isinstance(other, KeypadNode):
            return False
        return self.pos == other.pos and self.dir == other.dir
    
    def __lt__(self, other):
        self.pos < other.pos
    
    def __str__(self):
        return f'{self.pos}'
    
    def __repr__(self):
        return self.__str__()
    
    def __hash__(self):
        return hash(self.pos)

class Keypad():
    def out_of_bounds(self, i, j, matrix):
        return i < 0 or j < 0 or i >= len(matrix) or j >= len(matrix[0])

    def __init__(self, child_keypad: list[list[str]], parent_keypad: list[list[str]], name: str):
        self.child_keypad = child_keypad
        self.init_ck_arr()
        self.child_keypad_map = {child_keypad[i][j]: (i, j) for i in range(len(child_keypad)) for j in range(len(child_keypad[0]))}
        self.parent_keypad = parent_keypad
        self.parent_keypad_map = {parent_keypad[i][j]: (i, j) for i in range(len(parent_keypad)) for j in range(len(parent_keypad[0]))}
        self.current_position = self.child_keypad_map['A']
        self.name = name

    def init_ck_arr(self):
        self.ck_arr: list[list[list[KeypadNode]]] = []
        for i in range(len(self.child_keypad)):
            row = []
            for j in range(len(self.child_keypad[0])):
                nodes = []
                for dir in range(4):
                    if self.child_keypad[i][j] == '_':
                        nodes.append(None)
                    else:
                        nodes.append(KeypadNode((i, j), dir, self.child_keypad[i][j]))
                row.append(nodes)
            self.ck_arr.append(row)

    def path_to_moves(self, path: list[KeypadNode]) -> list[tuple[int, int]]:
        moves = []
        for i in range(1, len(path)):
            moves.append((path[i].pos[0] - path[i-1].pos[0], path[i].pos[1] - path[i-1].pos[1]))
        return moves

    def press(self, target: str) -> list[str]:
        x2, y2 = self.child_keypad_map[target]
        shortest_path = self.get_shortest_path(self.current_position, (x2, y2))
        moves = self.path_to_moves(shortest_path)
        presses = [move_dict[move] for move in moves] + ['A']
        print(f"{self} moved to {self.val_at((x2, y2))} in {presses}")
        self.current_position = (x2, y2)
        return presses
    
    def get_shortest_path(self, start: tuple[int, int], end: tuple[int, int]) -> list[tuple[int, int]]:
        start = self.ck_arr[start[0]][start[1]][0]
        open_set = []
        closed_list = []
        heapq.heappush(open_set, (0, start))
        came_from = {}
        g_cost = {}
        g_cost[(start.pos, start.dir)] = 0

        while len(open_set) > 0:
            current_node = open_set.pop(0)[1]
            closed_list.append(current_node)

            if current_node.pos == end:
                return self.reconstruct_path(came_from, current_node)

            for neighbor in self.get_neighbors(current_node):
                if neighbor in closed_list:
                    continue

                cost = self.calc_cost(current_node, neighbor)
                new_cost = g_cost[(current_node.pos, current_node.dir)] + cost

                if new_cost < g_cost.get((neighbor.pos, neighbor.dir), float('inf')):
                    g_cost[(neighbor.pos, neighbor.dir)] = new_cost
                    priority = new_cost + self.heuristic(neighbor, end)
                    heapq.heappush(open_set, (priority, neighbor))
                    came_from[(neighbor.pos, neighbor.dir)] = current_node
                
        return None
    
    def calc_cost(self, n1: KeypadNode, n2: KeypadNode):
        if abs(n1.pos[0]-n2.pos[0]) + abs(n1.pos[1]-n2.pos[1]) > 1:
            raise ValueError('Nodes must be adjacent')
        if n1.dir != n2.dir:
            return 3
        else:
            return 1

    def reconstruct_path(self, came_from: dict[tuple[tuple[int, int], int], KeypadNode], current: KeypadNode) -> list[KeypadNode]:
        total_path = [current]
        while (current.pos, current.dir) in came_from:
            current = came_from[(current.pos, current.dir)]
            total_path.append(current)
        return total_path[::-1]
    
    def heuristic(self, n1: KeypadNode, end: tuple[int, int]) -> int:
        # manhattan distance
        return abs(n1.pos[0] - end[0]) + abs(n1.pos[1] - end[1])

    def get_neighbors(self, node: KeypadNode) -> list[KeypadNode]:
        neighbors = []

        self_nodes = self.ck_arr[node.pos[0]][node.pos[1]] # list of nodes at this position (different directions)
        for self_node in self_nodes:
            if self_node.dir in range(4):
                neighbors.append(self_node)

        for mov in KeypadNode.moves:
            n_pos = node.pos[0] + mov[0], node.pos[1] + mov[1]
            if self.out_of_bounds(n_pos[0], n_pos[1], self.ck_arr):
                continue
            n_node = self.ck_arr[n_pos[0]][n_pos[1]][move_dir_map[mov]]
            if n_node is not None:
                neighbors.append(n_node)
            
        return neighbors
    
    def val_at(self, pos: tuple[int, int]) -> str:
        return self.ck_arr[pos[0]][pos[1]][0].val
                
    def __str__(self):
        return f"{self.name} -> {self.val_at(self.current_position)}"


In [3]:
numpad = Keypad(num_keypad, directional_keypad, 'numpad')
moves = numpad.press('0')
assert moves == ['<', 'A'], moves
assert numpad.current_position == (3, 1), numpad.current_position
moves = numpad.press('2')
assert moves == ['^', 'A'], moves
assert numpad.current_position == (2, 1)

dirpad = Keypad(directional_keypad, directional_keypad, 'dirpad')
moves = dirpad.press('<')
assert moves == ['v', '<', '<', 'A'], moves
assert dirpad.current_position == (1, 0), dirpad.current_position

numpad = Keypad(num_keypad, directional_keypad, 'numpad')
numpad.current_position = (3, 1)
moves = numpad.press('7')
assert moves == ['^', '^', '^', '<', 'A'], moves

dirpad = Keypad(directional_keypad, num_keypad, 'dirpad')
dirpad.current_position = (1, 0)
moves = dirpad.press('A')
assert moves == ['>', '>', '^', 'A'], moves

numpad -> A moved to 0 in ['<', 'A']
numpad -> 0 moved to 2 in ['^', 'A']
dirpad -> A moved to < in ['v', '<', '<', 'A']
numpad -> 0 moved to 7 in ['^', '^', '^', '<', 'A']
dirpad -> < moved to A in ['>', '>', '^', 'A']


In [4]:
doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')

sequence = "029A"
moves = [doorpad.press(target) for target in sequence]
moves = [move for sublist in moves for move in sublist]
moves = "".join(moves)

assert moves == '<A^A>^^AvvvA', moves

doorpad -> A moved to 0 in ['<', 'A']
doorpad -> 0 moved to 2 in ['^', 'A']
doorpad -> 2 moved to 9 in ['>', '^', '^', 'A']
doorpad -> 9 moved to A in ['v', 'v', 'v', 'A']


In [5]:
doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')
doorpadbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotpad')

# pipe the presses from the human to the door
sequence = "029A"

doorpad_moves = [doorpad.press(target) for target in sequence]
doorpad_moves = [move for sublist in doorpad_moves for move in sublist]

doorpadbot_moves = [doorpadbot.press(target) for target in doorpad_moves]
doorpadbot_moves = [move for sublist in doorpadbot_moves for move in sublist]
doorpadbot_moves = "".join(doorpadbot_moves)

assert doorpadbot_moves == 'v<<A>>^A<A>AvA^<AA>Av<AAA>^A', doorpadbot_moves

doorpad -> A moved to 0 in ['<', 'A']
doorpad -> 0 moved to 2 in ['^', 'A']
doorpad -> 2 moved to 9 in ['>', '^', '^', 'A']
doorpad -> 9 moved to A in ['v', 'v', 'v', 'A']
doorpadbotpad -> A moved to < in ['v', '<', '<', 'A']
doorpadbotpad -> < moved to A in ['>', '>', '^', 'A']
doorpadbotpad -> A moved to ^ in ['<', 'A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to > in ['v', 'A']
doorpadbotpad -> > moved to ^ in ['^', '<', 'A']
doorpadbotpad -> ^ moved to ^ in ['A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to v in ['v', '<', 'A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to A in ['>', '^', 'A']


In [6]:
doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')
doorpadbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotpad')
doorpadbotbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotbotpad')

sequence = "029A"

doorpad_moves = [doorpad.press(target) for target in sequence]
doorpad_moves = [move for sublist in doorpad_moves for move in sublist]

doorpadbot_moves = [doorpadbot.press(target) for target in doorpad_moves]
doorpadbot_moves = [move for sublist in doorpadbot_moves for move in sublist]

doorpadbotbot_moves = [doorpadbotbot.press(target) for target in doorpadbot_moves]
doorpadbotbot_moves = [move for sublist in doorpadbotbot_moves for move in sublist]
doorpadbotbot_moves = "".join(doorpadbotbot_moves)

sequence_num = int(sequence.replace('A', ''))
print(sequence_num)
print(doorpadbotbot_moves)
print(len(doorpadbotbot_moves))
print(f"complexity: {len(doorpadbotbot_moves)*sequence_num}")
assert len(doorpadbotbot_moves)*sequence_num == 1972, len(doorpadbotbot_moves)*sequence_num

doorpad -> A moved to 0 in ['<', 'A']
doorpad -> 0 moved to 2 in ['^', 'A']
doorpad -> 2 moved to 9 in ['>', '^', '^', 'A']
doorpad -> 9 moved to A in ['v', 'v', 'v', 'A']
doorpadbotpad -> A moved to < in ['v', '<', '<', 'A']
doorpadbotpad -> < moved to A in ['>', '>', '^', 'A']
doorpadbotpad -> A moved to ^ in ['<', 'A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to > in ['v', 'A']
doorpadbotpad -> > moved to ^ in ['^', '<', 'A']
doorpadbotpad -> ^ moved to ^ in ['A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to v in ['v', '<', 'A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to A in ['>', '^', 'A']
doorpadbotbotpad -> A moved to v in ['v', '<', 'A']
doorpadbotbotpad -> v moved to < in ['<', 'A']
doorpadbotbotpad -> < moved to < in ['A']
doorpadbotbotpad -> < moved to A in ['>', '>', '^', 'A']
doorpadbotbotpad -> A moved to > in ['v', 'A']
doorpadbotbotpad -> > moved to > in

In [7]:
def calc_complexity(sequence: list[str]) -> int:
    doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')
    doorpadbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotpad')
    doorpadbotbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotbotpad')

    doorpad_moves = [doorpad.press(target) for target in sequence]
    doorpad_moves = [move for sublist in doorpad_moves for move in sublist]

    doorpadbot_moves = [doorpadbot.press(target) for target in doorpad_moves]
    doorpadbot_moves = [move for sublist in doorpadbot_moves for move in sublist]

    doorpadbotbot_moves = [doorpadbotbot.press(target) for target in doorpadbot_moves]
    doorpadbotbot_moves = [move for sublist in doorpadbotbot_moves for move in sublist]
    doorpadbotbot_moves = "".join(doorpadbotbot_moves)

    sequence_num = int(sequence.replace('A', ''))
    return (doorpadbotbot_moves, sequence_num, len(doorpadbotbot_moves)*sequence_num)

def run(sequences: list[str]) -> int:
    complexity_sum = 0
    sequence_nums = []
    move_sets = []

    for sequence in sequences:
        moves, sequence_num, com = calc_complexity(sequence)
        complexity_sum += com
        sequence_nums.append(sequence_num)
        move_sets.append(moves)

    print(f"complexity sum: {complexity_sum}")
    for i, _ in enumerate(sequences):
        print(sequences[i], end=': ')
        print(f"len: {len(move_sets[i])}, ", end=' ')
        print(f"sequence_num: {sequence_nums[i]}, ", end=' ')
        print(f"complexity: {len(move_sets[i])*sequence_nums[i]}")
        print(f"moves: {move_sets[i]}")
        print()

    return complexity_sum

In [8]:
sequences = ["029A","980A","179A","456A","379A"]
complexity_sum = run(sequences)
assert complexity_sum == 126384, complexity_sum

doorpad -> A moved to 0 in ['<', 'A']
doorpad -> 0 moved to 2 in ['^', 'A']
doorpad -> 2 moved to 9 in ['>', '^', '^', 'A']
doorpad -> 9 moved to A in ['v', 'v', 'v', 'A']
doorpadbotpad -> A moved to < in ['v', '<', '<', 'A']
doorpadbotpad -> < moved to A in ['>', '>', '^', 'A']
doorpadbotpad -> A moved to ^ in ['<', 'A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to > in ['v', 'A']
doorpadbotpad -> > moved to ^ in ['^', '<', 'A']
doorpadbotpad -> ^ moved to ^ in ['A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to v in ['v', '<', 'A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to A in ['>', '^', 'A']
doorpadbotbotpad -> A moved to v in ['v', '<', 'A']
doorpadbotbotpad -> v moved to < in ['<', 'A']
doorpadbotbotpad -> < moved to < in ['A']
doorpadbotbotpad -> < moved to A in ['>', '>', '^', 'A']
doorpadbotbotpad -> A moved to > in ['v', 'A']
doorpadbotbotpad -> > moved to > in

In [9]:
sequences = ["670A","974A","638A","319A","508A"]
complexity_sum = run(sequences)

doorpad -> A moved to 6 in ['^', '^', 'A']
doorpad -> 6 moved to 7 in ['<', '<', '^', 'A']
doorpad -> 7 moved to 0 in ['>', 'v', 'v', 'v', 'A']
doorpad -> 0 moved to A in ['>', 'A']
doorpadbotpad -> A moved to ^ in ['<', 'A']
doorpadbotpad -> ^ moved to ^ in ['A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to < in ['v', '<', '<', 'A']
doorpadbotpad -> < moved to < in ['A']
doorpadbotpad -> < moved to ^ in ['>', '^', 'A']
doorpadbotpad -> ^ moved to A in ['>', 'A']
doorpadbotpad -> A moved to > in ['v', 'A']
doorpadbotpad -> > moved to v in ['<', 'A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to v in ['A']
doorpadbotpad -> v moved to A in ['>', '^', 'A']
doorpadbotpad -> A moved to > in ['v', 'A']
doorpadbotpad -> > moved to A in ['^', 'A']
doorpadbotbotpad -> A moved to < in ['v', '<', '<', 'A']
doorpadbotbotpad -> < moved to A in ['>', '>', '^', 'A']
doorpadbotbotpad -> A moved to A in ['A']
doorpadbotbotpad -> A moved to > in ['v', 'A']